In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from astropy.table import Table
from omnitool.literature_values import *

In [25]:
#Read in Jie Yu
def read_yu18():
    sfile = '/home/oliver/PhD/Catalogues/RC_catalogues/Yu+18_table1.txt'
#     cols = ['KICID','numax','err','dnu','err.1']
    yu18_1 = pd.read_csv(sfile, sep='|')

    sfile = '/home/oliver/PhD/Catalogues/RC_catalogues/Yu+18_table2.txt'
#     cols = ['KICID','Teff','err','logg','err.1_y','Fe/H','err.2','EvoPhase']
    yu18_2 = pd.read_csv(sfile, sep='|')
    yu18 = pd.merge(yu18_1, yu18_2, on='KICID',how='left')
    yu18.rename(columns={'EvoPhase':'stage',
                        'err_x':'numax_err',
                        'err.1_x':'dnu_err',
                        'err_y':'Teff_err',
                         'Fe/H':'[Fe/H]',
                        'err.2':'[Fe/H]_err',
                        'logg':'yu_logg',
                        'err.1_y':'yu_logg_err',
                        'err.3_y':'M_err',
                        'err.4_y':'R_err'},inplace=True) #For consistency
    return yu18

In [26]:
ydf = read_yu18()
ydf.head(2)
print(list(ydf))

['KICID', 'kepmag', 'Length_Quarters', 'Length_days', 'numax', 'numax_err', 'dnu', 'dnu_err', 'amplitude', 'err.2_x', 'fwhm', 'err.3_x', 'maxbg', 'err.4_x', 'Unnamed: 14', 'Teff', 'Teff_err', 'yu_logg', 'yu_logg_err', '[Fe/H]', 'err.2_y', 'M_noCorrection', 'M_err', 'R_noCorrection', 'R_err', 'M_RGB', 'err.5', 'R_RGB', 'err.6', 'M_Clump', 'err.7', 'R_Clump', 'err.8', 'stage', 'Unnamed: 20']


In [27]:
data = Table.read('../data/KepxDR2/kepler_dr2_1arcsec.fits', format='fits')
kdf = data.to_pandas()
kdf.rename(columns={'kepid':'KICID'},inplace=True)
kdf.head(2)

,designation,source_id,gaia_ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,...,mass_err2,prov_sec,nconfp,nkoi,ntce,jmag,hmag,kmag,planet?,kepler_gaia_ang_dist
0,Gaia DR2 2050233807328471424,2.050234e+18,2015.5,291.038681,0.035878,36.598031,0.047832,1.493916,0.048328,30.912294,...,-0.475,DSEP,0.0,0.0,0.0,10.126,9.667,9.559,none,0.381502
1,Gaia DR2 2050233601176543104,2.050234e+18,2015.5,291.043057,0.019919,36.593767,0.027553,2.678194,0.027151,98.642212,...,-0.233,DSEP,0.0,0.0,1.0,11.656,11.193,11.094,none,0.158389


In [28]:
df = pd.merge(ydf, kdf, on='KICID',how='left')
print(len(df))

16135


In [ ]:
sns.jointplot(df.Teff, df.teff)
plt.show()
sns.jointplot(df['[Fe/H]'], df.feh)
plt.show()
sns.jointplot(df.yu_logg,df.logg)
plt.show()

In [ ]:
df.to_csv('../data/KepxDR2/xyu18.csv')

In [29]:
'''<<<SELECT RC ONLY STARS>>>'''
print('Selecting only RC stars & existing parallax')
print('Before: '+str(len(df)))
rcdf = df[df.stage==2]
rcdf = rcdf[np.isfinite(rcdf.parallax)]
print('After: '+str(len(rcdf)))

# print('Dropping duplicates')
# print('Before: '+str(len(rcdf)))
# rcdf.drop_duplicates('KICID',inplace=True)
# rcdf = rcdf.reindex()
# print('After: '+str(len(rcdf)))

# rcdf.to_csv('../data/KepxDR2/RCxyu18.csv')

Selecting only RC stars & existing parallax
Before: 16135
After: 7692


In [14]:
#Now lets read in the KICID and relevant information from APOKASC
sfile = '/home/oliver/PhD/Catalogues/APOKASC/APOKASC_cat_v3.6.5.txt'
cols= ['KEPLER_ID',\
      'J_MAG_2M', 'J_MAG_ERR', 'H_MAG_2M', 'H_MAG_ERR',\
       'K_MAG_2M', 'K_MAG_ERR']

apokasc = pd.read_csv(sfile, usecols=cols, skiprows=644,sep='\s+')
apokasc.rename(columns={'KEPLER_ID':'KICID'},inplace=True)

apokasc.head(2)

,KICID,J_MAG_2M,J_MAG_ERR,H_MAG_2M,H_MAG_ERR,K_MAG_2M,K_MAG_ERR
0,10000207,10.811,0.021,10.261,0.017,10.172,0.012
1,10000547,10.712,0.021,10.247,0.019,10.161,0.014


In [33]:
from omnitool import spyglass
sg = spyglass()
sg.pass_position(rcdf.ra, rcdf.dec, frame='icrs')
sg.pass_distance(rcdf.r_est)
rcdf['Ebv'] = sg.get_Ebv()

In [34]:
fdf = pd.merge(rcdf, apokasc, on='KICID',how='left')
print(len(np.isfinite(apokasc.K_MAG_2M)))
print(len(np.isfinite(fdf.K_MAG_2M)))
print('Saving')
fdf.to_csv('../data/KepxDR2/RCxyu18_wdupes.csv')

11876
7692
Saving


In [35]:
print(fdf.Ebv)
import sys
sys.exit()

0       0.102378
1       0.093784
2       0.114852
3       0.075924
4       0.261683
5       0.158485
6       0.104443
7       0.043564
8       0.049708
9       0.067402
10      0.088036
11      0.087720
12      0.081256
13      0.077898
14      0.084563
15      0.113927
16      0.147160
17      0.085268
18      0.150913
19      0.071231
20      0.062592
21      0.136890
22      0.133993
23      0.101761
24      0.067036
25      0.077226
26      0.110622
27      0.121745
28      0.123917
29      0.164447
          ...   
7662    0.045896
7663    0.043901
7664    0.039366
7665    0.047440
7666    0.047429
7667    0.037715
7668    0.100974
7669    0.061386
7670    0.061016
7671    0.050493
7672    0.076319
7673    0.083830
7674    0.083830
7675    0.089298
7676    0.078941
7677    0.086859
7678    0.041475
7679    0.052030
7680    0.051368
7681    0.065243
7682    0.043869
7683    0.072230
7684    0.066825
7685    0.065900
7686    0.069245
7687    0.073474
7688    0.043461
7689    0.0783

SystemExit: 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
